### Simple GEN-AI app using Langchain

Load all the environment variables

In [26]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["LANGSMITH_API_KEY"] = os.getenv("LANGSMITH_API_KEY")
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = os.getenv("LANGSMITH_PROJECT")
os.environ["LANGSMITH_ENDPOINT"] = os.getenv("LANGSMITH_ENDPOINT")

Scrape data from a website

In [27]:
from langchain_community.document_loaders import WebBaseLoader
import bs4

class_label = "beebom-single-content-container"
web_url = "https://beebom.com/valorant-characters-agents-abilities"

loader = WebBaseLoader(web_paths=[web_url],
                bs_kwargs={
                    "parse_only": bs4.SoupStrainer(class_=class_label),
                })
docs = loader.load()

Divide the raw data from the website into chunks

In [28]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
)

splitted_docs = text_splitter.split_documents(docs)

Generate embeddings of the documents

In [29]:
from langchain_huggingface import HuggingFaceEmbeddings

EMBEDDING_MODEL = "BAAI/bge-small-en-v1.5"
embedder = HuggingFaceEmbeddings(
    model_name=EMBEDDING_MODEL,
    multi_process=True,
    model_kwargs={
        "device": "cuda",
    },
    encode_kwargs={
        "normalize_embeddings": True,
    }
)

Store the embeddings in the vector database FAISS

In [30]:
from langchain_community.vectorstores import FAISS

vectorstore_db = FAISS.from_documents(
    splitted_docs,
    embedder
)

Query the vectorstore DB

In [31]:
query = "What are the abilities of Killjoy?"
result = vectorstore_db.similarity_search(query, k=3)

Add retrieval chain to the LLM

In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

prompt = ChatPromptTemplate.from_template(
    """
    You are a helpful assistant. Use the following pieces of context wrapped within <context>..</context> to answer the question.
    <context>
    {context}
    </context>
    Question: {input}
    """
)

llm = ChatGroq(
    model_name="llama-3.3-70b-versatile",
    temperature=0.7
)

retriever = vectorstore_db.as_retriever(
    search_type="similarity",
    search_kwargs={
        "k": 5,
    }
)

document_chain = create_stuff_documents_chain(
    llm=llm,
    prompt=prompt)


retrieval_chain = create_retrieval_chain(retriever,document_chain)

In [38]:
response = retrieval_chain.invoke({"input": query})

In [39]:
response

{'input': 'What are the abilities of Killjoy?',
 'context': [Document(id='e59dc951-2377-4961-901a-06637e78b3eb', metadata={'source': 'https://beebom.com/valorant-characters-agents-abilities'}, page_content='12. Killjoy\n\nImage Credit: Riot Games/ Valorant (screenshot by Ishan Adhikary/ Beebom)\n\nRole – Sentinel\n\nKilljoy is more than just one agent in Valorant. Yes, put her Turret in a location and it will do its part just like another agent in your team. She is a great defensive agent in the game with lots of shutdown tools.'),
  Document(id='64516357-de5e-43b7-816e-4c96de6a0629', metadata={'source': 'https://beebom.com/valorant-characters-agents-abilities'}, page_content='Alarmbot (Q) – Put a scanner bot in place. The bot stays hidden until it scans an enemy. Bot chases down an enemy after finding it and explodes on contact. After the explosion, the bot makes the affected enemy vulnerable for a brief duration.\nNanoswarm (C) – Killjoy can place swarm grenades anywhere on the map. 